# 문제

간단한 은행 컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

은행 컨트랙의 함수는 다음과 같이 만드세요.

* 입금액을 정해서 입금. 입금액을 매개변수로 넣어준다.

	function deposit(uint amount) public payable
    

* 전액 인출 함수. 인출 금액을 정하기 않아도 되므로 함수 인자가 없다.

	function widthdrawAll() public
    

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)
    

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable

In [1]:
%cd C:\Users\jeh\Code\201910835

C:\Users\jeh\Code\201910835


In [2]:
%%writefile src/week11/SimpleBank.sol

pragma solidity ^0.6.0;

contract SimpleBank{
    address payable owner;
    uint balance;
    uint inc;
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function widthdrawAll() public {
        inc = owner.balance;
        owner.transfer(balance);
        balance -= balance;
        inc = owner.balance-inc;
    }
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balance);
    }
    function forwardTo(address payable _receiver) public payable {
        inc = address(_receiver).balance;
        _receiver.transfer(msg.value);
        inc = address(_receiver).balance-inc;
    }
    function getBalanceInc() public view returns(uint) {
        return inc;
    }
}

Writing src/week11/SimpleBank.sol


In [3]:
!solc --gas --abi --bin src/week11/SimpleBank.sol


======= src/week11/SimpleBank.sol:SimpleBank =======
Gas estimation:
construction:
   41090 + 162600 = 203690
external:
   deposit(uint256):	21125
   forwardTo(address):	infinite
   getBalance():	1027
   getBalanceInc():	1057
   widthdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055504760018190555061032d806100676000396000f3fe60806040526004361061004a5760003560e01c806312065fe01461004f57806327d8ad88146100815780633c459375146100c557806361c06694146100dc578063b6b55f2514610107575b600080fd5b34801561005b57600080fd5b50610064610135565b604051808381526020018281526020019250505060405180910390f35b6100c36004803603602081101561009757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610144565b005b3480156100d157600080fd5b506100da6101ce565b005b3480156100e857600080fd5b506100f16102ce565b6040518082815260200191505060405180910390f35b610

In [4]:
!node src/myAddr.js

[
  '0x801e2e6b3de149B674467aDfE924399082E7c6f8',
  '0xfda5E0295edB1b8a54fc83B75b34d7558a7b4D2A',
  '0x870B70C1af7f7C05994C26a6e994BB9Cf3EeA781',
  '0x640C84442Cb621caBa1e7481421fE29594781c41',
  '0xBB771Ad1AdC710ed98d928BEae0854479f2e954E',
  '0x173960c4E2008a66A3B9341fFaD163981e3dc961',
  '0x4ED70c3239deCD3480551A05e67F9050800B3221',
  '0xFc108897CD8e056E5532263Ba304C3c29f9B935B',
  '0x77FE44A13b61F08F72006689Ae58718B1af3f36b',
  '0xf6e688EeeC3f083984995c94Fbfa0ADe13FA44e4'
]


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


## ganache에 배포

In [5]:
%%writefile src/week11/SimpleBankDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
}
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055504760018190555061032d806100676000396000f3fe60806040526004361061004a5760003560e01c806312065fe01461004f57806327d8ad88146100815780633c459375146100c557806361c06694146100dc578063b6b55f2514610107575b600080fd5b34801561005b57600080fd5b50610064610135565b604051808381526020018281526020019250505060405180910390f35b6100c36004803603602081101561009757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610144565b005b3480156100d157600080fd5b506100da6101ce565b005b3480156100e857600080fd5b506100f16102ce565b6040518082815260200191505060405180910390f35b6101336004803603602081101561011d57600080fd5b81019080803590602001909291905050506102d8565b005b60008047600154915091509091565b8073ffffffffffffffffffffffffffffffffffffffff16316002819055508073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101a8573d6000803e3d6000fd5b506002548173ffffffffffffffffffffffffffffffffffffffff16310360028190555050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16316002819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc6001549081150290604051600060405180830381858888f19350505050158015610276573d6000803e3d6000fd5b506001546001600082825403925050819055506002546000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163103600281905550565b6000600254905090565b8034146102e457600080fd5b806001600082825401925050819055505056fea264697066735822122071ec422e478023e14b73906bf3169e9e900e8fda7cc2d402c9aef26b70ae7d8a64736f6c63430006040033";
var _contract = new web3.eth.Contract(_abiArray);
_contract
.deploy({data:"0x"+_bin})
    .send({from:"0x801e2e6b3de149B674467aDfE924399082E7c6f8",gas:1000000})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Writing src/week11/SimpleBankDeploy.js


In [6]:
!node src/week11/SimpleBankDeploy.js

0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 입금 11111 wei, 222 wei

In [7]:
%%writefile src/week11/SimpleBankUse1_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, '0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2');

bank.methods.deposit(11111).send({from:"0x801e2e6b3de149B674467aDfE924399082E7c6f8",gas:80000,value:11111});
bank.methods.deposit(222).send({from:"0x801e2e6b3de149B674467aDfE924399082E7c6f8",gas:80000,value:222});

Writing src/week11/SimpleBankUse1_1.js


In [8]:
!node src/week11/SimpleBankUse1_1.js

web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 잔고 11333 wei 출력

In [9]:
%%writefile src/week11/SimpleBankUse1_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, '0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2');

bank.methods.getBalance().call().then(console.log);

Writing src/week11/SimpleBankUse1_2.js


In [10]:
!node src/week11/SimpleBankUse1_2.js

Result { '0': '11333', '1': '11333' }


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 자신의 2번째 계정으로 계좌이체 333 wei

In [11]:
%%writefile src/week11/SimpleBankUse2_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, '0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2');

bank.methods.forwardTo("0xfda5E0295edB1b8a54fc83B75b34d7558a7b4D2A").send({from:"0x801e2e6b3de149B674467aDfE924399082E7c6f8",gas:100000,value:333});

Writing src/week11/SimpleBankUse2_1.js


In [12]:
!node src/week11/SimpleBankUse2_1.js

web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 컨트랙 잔고 출력, 자신의 2번째 계정 잔고 증가분 출력

In [13]:
%%writefile src/week11/SimpleBankUse2_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, '0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2');

bank.methods.getBalance().call().then(console.log);
bank.methods.getBalanceInc().call().then(console.log);

Writing src/week11/SimpleBankUse2_2.js


In [14]:
!node src/week11/SimpleBankUse2_2.js

Result { '0': '11333', '1': '11333' }
333


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 전액 인출

In [15]:
%%writefile src/week11/SimpleBankUse3_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, '0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2');

bank.methods.widthdrawAll().send({from:"0x801e2e6b3de149B674467aDfE924399082E7c6f8"});

Writing src/week11/SimpleBankUse3_1.js


In [16]:
!node src/week11/SimpleBankUse3_1.js

web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 자신의 잔고 증가분 출력

In [17]:
%%writefile src/week11/SimpleBankUse3_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceInc","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, '0x1EFCc13F0EF9D6C7C318A6F99B3911432FfE1AF2');

bank.methods.getBalanceInc().call().then(console.log);

Writing src/week11/SimpleBankUse3_2.js


In [18]:
!node src/week11/SimpleBankUse3_2.js

11333


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


## geth에 배포

In [2]:
%%writefile src/week11/SimpleBankGeth.sol

pragma solidity 0.5.0;

contract SimpleBank{
    address payable owner;
    uint balance;
    uint inc;
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    function deposit() public payable {
        balance += msg.value;
    }
    function widthdrawAll() public {
        inc = owner.balance;
        owner.transfer(balance);
        balance -= balance;
        inc = owner.balance-inc;
    }
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balance);
    }
    function forwardTo(address payable _receiver) public payable {
        inc = address(_receiver).balance;
        _receiver.transfer(msg.value);
        inc = address(_receiver).balance-inc;
    }
    function getBalanceInc() public view returns(uint) {
        return inc;
    }
}

Overwriting src/week11/SimpleBankGeth.sol


In [3]:
%%writefile src/week11/SimpleBankDeploy_geth.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
}
var _abi=
[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055503073ffffffffffffffffffffffffffffffffffffffff163160018190555061035f8061007e6000396000f3fe60806040526004361061006d576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe01461007257806327d8ad88146100a45780633c459375146100e857806361c06694146100ff578063b6b55f251461012a575b600080fd5b34801561007e57600080fd5b50610087610158565b604051808381526020018281526020019250505060405180910390f35b6100e6600480360360208110156100ba57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061017e565b005b3480156100f457600080fd5b506100fd610208565b005b34801561010b57600080fd5b50610114610308565b6040518082815260200191505060405180910390f35b6101566004803603602081101561014057600080fd5b8101908080359060200190929190505050610312565b005b6000803073ffffffffffffffffffffffffffffffffffffffff1631600154915091509091565b8073ffffffffffffffffffffffffffffffffffffffff16316002819055508073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101e2573d6000803e3d6000fd5b506002548173ffffffffffffffffffffffffffffffffffffffff16310360028190555050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16316002819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc6001549081150290604051600060405180830381858888f193505050501580156102b0573d6000803e3d6000fd5b506001546001600082825403925050819055506002546000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163103600281905550565b6000600254905090565b803414151561032057600080fd5b806001600082825401925050819055505056fea165627a7a72305820fd4bba40bedffbaae7c7a5fe8e6d29bfcd9e8e0b8fa1c65fb34bfb87c65c6a8a0029";
var _contract = new web3.eth.Contract(_abi);
_contract
.deploy({data:"0x"+_bin})
    .send({from:"0xe3f6601a5f2f59e5ad17367bcb5e4d6bd1cf5fcd",gas:1000000})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/week11/SimpleBankDeploy_geth.js


In [4]:
!node src/week11/SimpleBankDeploy_geth.js

0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 입금 11111 wei, 222 wei

In [5]:
%%writefile src/week11/SimpleBankUse_geth_1_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var bank = new web3.eth.Contract(_abiArray, '0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf');

bank.methods.deposit(11111).send({from:"0xe3f6601a5f2f59e5ad17367bcb5e4d6bd1cf5fcd",gas:80000,value:11111});
bank.methods.deposit(222).send({from:"0xe3f6601a5f2f59e5ad17367bcb5e4d6bd1cf5fcd",gas:80000,value:222});

Overwriting src/week11/SimpleBankUse_geth_1_1.js


In [6]:
!node src/week11/SimpleBankUse_geth_1_1.js

web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 잔고 11333 wei 출력

In [7]:
%%writefile src/week11/SimpleBankUse_geth_1_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var bank = new web3.eth.Contract(_abiArray, '0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf');

bank.methods.getBalance().call().then(console.log);

Overwriting src/week11/SimpleBankUse_geth_1_2.js


In [8]:
!node src/week11/SimpleBankUse_geth_1_2.js

Result { '0': '11333', '1': '11333' }


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 자신의 2번째 계정으로 계좌이체 333 wei

In [11]:
%%writefile src/week11/SimpleBankUse_geth_2_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var bank = new web3.eth.Contract(_abiArray, '0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf');

bank.methods.forwardTo("0xfbf867b98c22d97ef1b8f222842376ea7acba7f3").send({from:"0xe3f6601a5f2f59e5ad17367bcb5e4d6bd1cf5fcd",gas:100000,value:333});

Overwriting src/week11/SimpleBankUse_geth_2_1.js


In [13]:
!node src/week11/SimpleBankUse_geth_2_1.js

web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 컨트랙 잔고 출력, 자신의 2번째 계정 잔고 증가분 출력

In [14]:
%%writefile src/week11/SimpleBankUse_geth_2_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var bank = new web3.eth.Contract(_abiArray, '0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf');

bank.methods.getBalance().call().then(console.log);
bank.methods.getBalanceInc().call().then(console.log);

Writing src/week11/SimpleBankUse_geth_2_2.js


In [15]:
!node src/week11/SimpleBankUse_geth_2_2.js

Result { '0': '11333', '1': '11333' }
333


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 전액 인출

In [16]:
%%writefile src/week11/SimpleBankUse_geth_3_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var bank = new web3.eth.Contract(_abiArray, '0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf');

bank.methods.widthdrawAll().send({from:"0xe3f6601a5f2f59e5ad17367bcb5e4d6bd1cf5fcd"});

Writing src/week11/SimpleBankUse_geth_3_1.js


In [17]:
!node src/week11/SimpleBankUse_geth_3_1.js

web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.


### 자신의 잔고 증가분 출력

In [18]:
%%writefile src/week11/SimpleBankUse_geth_3_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[
	{
		"constant": true,
		"inputs": [],
		"name": "getBalance",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			},
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "_receiver",
				"type": "address"
			}
		],
		"name": "forwardTo",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [],
		"name": "widthdrawAll",
		"outputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"constant": true,
		"inputs": [],
		"name": "getBalanceInc",
		"outputs": [
			{
				"name": "",
				"type": "uint256"
			}
		],
		"payable": false,
		"stateMutability": "view",
		"type": "function"
	},
	{
		"constant": false,
		"inputs": [
			{
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "deposit",
		"outputs": [],
		"payable": true,
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [],
		"payable": false,
		"stateMutability": "nonpayable",
		"type": "constructor"
	}
];
var bank = new web3.eth.Contract(_abiArray, '0xf20Fc2EB699aa490a0Ea312fe383DbaEda7B1eBf');

bank.methods.getBalanceInc().call().then(console.log);

Writing src/week11/SimpleBankUse_geth_3_2.js


In [19]:
!node src/week11/SimpleBankUse_geth_3_2.js

11333


web3-shh package will be deprecated in version 1.3.5 and will no longer be supported.
web3-bzz package will be deprecated in version 1.3.5 and will no longer be supported.
